## 内核EPOLL模型讲解
+ 首先我们来定义流的概念，一个流可以是文件，socket，pipe等可以进行I/O操作的内核对象。不管是文件，还是套接字(socket)，还是管道(pipe)，我们都可以把他们看作流。之后我们来讨论I/O操作，通过read，我们可以从流中读入数据；通过write，我们可以往流中写入数据。现在假定1种情形，我们需要从流中读数据，但是流中还没有数据，（典型的例子为，客户端要从socket读数据，但是服务器端还没有把数据传回来），这时候该怎么办？
+ 阻塞：阻塞是个什么概念呢？比如某个时候你在等快递，但是你还不知道快递什么时候过来，而且你也没有别的事可以干（或者说接下来的事要等快递来了才能做）；那么你可以去睡觉了，因为你知道快递把货送来时一定会给你打电话（假定一定能叫醒你）。
+ 非阻塞忙轮询：接着上面等快递的例子，如果用忙轮询的方法，那么你需要知道快递员的手机号，然后每分钟给他打个电话：“你到了没？”
+ 很明显一般人不会用第二种做法，不仅显得无脑，浪费话费不说，还占用了快递员大量的时间。
+ 大部分程序也不会用第二种做法，因为第一种方法经济而简单，经济是指消耗很少的CPU时间，如果线程睡眠了，就掉出了系统的调度队列，暂时不会去瓜分CPU宝贵的时间片。
+ 为了了解阻塞是如何进行的，我们来讨论缓冲区，以及内核缓冲区，最终把I/O事件解释清楚。缓冲区的引入是为了减少频繁I/O操作而引起频繁的系统调用（你知道它很慢的），当你操作一个流时，更多的是以缓冲区为单位进行操作，这是相对于用户空间而言。对于内核来说，也需要缓冲区。
+ 假设有一个管道，进程A为管道的写入方，B为管道的读出方。假设一开始内核缓冲区是空的，B作为读出方，被阻塞着。然后首先A往管道写入，这时候内核缓冲区由空的状态变到非空状态，内核就会产生一个事件告诉B该醒来了，这个事件姑且称之为“缓冲区非空”。但是“缓冲区非空”事件通知B后，B却还没有读出数据；且内核许诺了不能把写入管道中的数据丢掉这个时候，Ａ写入的数据会滞留在内核缓冲区中，如果内核也缓冲区满了，B仍未开始读数据，最终内核缓冲区会被填满，这个时候会产生一个I/O事件，告诉进程A，你该等等（阻塞）了，我们把这个事件定义为“缓冲区满”。假设后来Ｂ终于开始读数据了，于是内核的缓冲区空了出来，这时候内核会告诉A，内核缓冲区有空位了，你可以从长眠中醒来了，继续写数据了，我们把这个事件叫做“缓冲区非满”。也许事件Y1已经通知了A，但是A也没有数据写入了，而Ｂ继续读出数据，知道内核缓冲区空了。这个时候内核就告诉B，你需要阻塞了！，我们把这个时间定为“缓冲区空”。
+ 这四种情形涵盖了四个I/O事件，内核缓冲区满，内核缓冲区空，内核缓冲区非空，内核缓冲区非满。这四个I/O事件是进行阻塞同步的根本。（如果不能理解“同步”是什么概念，请学习操作系统的锁，信号量，条件变量等任务同步方面的相关知识）。

##  python中的epoll
### 1、触发方式：
+ 边缘触发/水平触发，只适用于Unix/Linux操作系统

### 2、一般步骤
+ 1.Create an epoll object——创建1个epoll对象
+ 2.Tell the epoll object to monitor specific events on specific sockets——告诉epoll对象，在指定的socket上监听指定的事件
+ 3.Ask the epoll object which sockets may have had the specified event since the last query——询问epoll对象，从上次查询以来，哪些socket发生了哪些指定的事件
+ 4.Perform some action on those sockets——在这些socket上执行一些操作
+ 5.Tell the epoll object to modify the list of sockets and/or events to monitor——告诉epoll对象，修改socket列表和（或）事件，并监控
+ 6.Repeat steps 3 through 5 until finished——重复步骤3-5，直到完成
+ 7.Destroy the epoll object——销毁epoll对象

### 3、相关用法
+ import select 导入select模块
+ epoll = select.epoll() 创建一个epoll对象
+ epoll.register(文件句柄,事件类型) 注册要监控的文件句柄和事件
### 事件类型:
+ 　　select.EPOLLIN    可读事件
+ 　　select.EPOLLOUT   可写事件
+ 　　select.EPOLLERR   错误事件
+ 　　select.EPOLLHUP   客户端断开事件

+ epoll.unregister(文件句柄)   销毁文件句柄 
+ epoll.poll(timeout)  当文件句柄发生变化，则会以列表的形式主动报告给用户进程,timeout为超时时间，默认为-1，即一直等待直到文件句柄发生变化，如果指定为1那么epoll每1秒汇报一次当前文件句柄的变化情况，如果无变化则返回空 
+ epoll.fileno() 返回epoll的控制文件描述符(Return the epoll control file descriptor)
+ epoll.modfiy(fineno,event) fineno为文件描述符 event为事件类型  作用是修改文件描述符所对应的事件
+ epoll.fromfd(fileno) 从1个指定的文件描述符创建1个epoll对象
+ epoll.close()   关闭epoll对象的控制文件描述符